In [ ]:
import os
import glob
import pickle
import pandas as pd
import numpy as np
from dask.diagnostics import ProgressBar
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell
import pandas as pd
import seaborn as sns

/Users/TenzinLhakhang/Desktop/scenic/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [6]:

DATA_FOLDER="tmp"
RESOURCES_FOLDER="resources"
DATABASE_FOLDER = "databases/"
SCHEDULER="123.122.8.24:8786"
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*.feather")
MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")
MM_TFS_FNAME = os.path.join(RESOURCES_FOLDER, 'mm_tfs.txt')
SC_EXP_FNAME = os.path.join(RESOURCES_FOLDER, "GSE60361_C1-3005-Expression.txt")
REGULONS_FNAME = os.path.join(DATA_FOLDER, "regulons.p")
MOTIFS_FNAME = os.path.join(DATA_FOLDER, "motifs.csv")


import pandas as pd

x = pd.read_table('GSE60361_C1-3005-Expression.txt')
x.index = x['cell_id']
del x['cell_id']
print(x.head())

          1772071015_C02  1772071017_G12  1772071017_A05  1772071014_B06  \
cell_id                                                                    
Tspan12                0               0               0               3   
Tshz1                  3               1               0               2   
Fnbp1l                 3               1               6               4   
Adamts15               0               0               0               0   
Cldn12                 1               1               1               0   

          1772067065_H06  1772071017_E02  1772067065_B07  1772067060_B09  \
cell_id                                                                    
Tspan12                0               0               3               0   
Tshz1                  2               2               2               1   
Fnbp1l                 1               2               1               0   
Adamts15               0               0               0               0   
Cldn12     

In [31]:
tf_raw = pd.read_csv('transcriptionfactor/known.tf.txt', delimiter='\t', error_bad_lines=False, encoding='ISO-8859-1')
tfs = tf_raw[['Gene ID', 'Evidence Strength']].drop_duplicates().dropna()
tfs['ID'] = list(map(int, tfs['Gene ID']))
tfs['ID'] = tfs['ID'].astype(str)

b'Skipping line 475: expected 12 fields, saw 13\n'


In [63]:
import mygene as mg

mg = mygene.MyGeneInfo()
entrez_to_symbol = mg.querymany(tfs['ID'], scopes='entrezgene', fields='symbol', species='mouse')
symbols = [row['symbol'] for row in entrez_to_symbol]
df_symbols = pd.DataFrame(symbols)
df_symbols.to_csv('resources/mm_tfs.txt',index=False,header=False)


querying 1-544...done.
Finished.


,0
0,Nr0b1
1,Ahr
2,Aire
3,Nr2f2
4,Ar
5,Phox2a
6,Arnt
7,Arnt2
8,Arntl
9,Arx
